# Arduino

Contains various radial map and driver versions for Borjas & Amies 1994

In [78]:
n = 1000 # Driver nPoints
f = 1 # Load Factor (scalar)

In [69]:
def BSRadialMap(dStrain, StaticParam, CurStress, CurState):
    # Radial Return Algorithm for J2 Bounding Surface Plasticity Model 
    # Borja and Amies 1994
    # Written by Pedro Arduino
    # Copyright - Arduino Computational Geomechanics Group
    # March, 2019
    # Ported into Python/Jupyter Notebook by Justin Bonus
    # July, 2019
    #
    # Input:
    #    dStrain               ... Strain differential, tn to tn+1
    #    StaticParam.E         ... Young's modulus
    #    StaticParam.v         ... poissons ratio
    #    StaticParam.G         ... Shear modulus
    #    StaticParam.K         ... Bulk modulus
    #    StaticParam.hh        ... Kinematic Hardening parameter
    #    StaticParam.mm        ... Kinematic hardening parameter
    #    StaticParam.beta      ... Integration parameter (0=expl, 1=impl)
    #    StaticParam. RR       ... Bounding surface radius
    #    CurStress             ... Stress at tn
    #    CurState.eP           ... Plastic strain at tn
    #    CurState.alphaISO     ... Isotropic internal variable at tn
    ##   CurState.alphaKIN     ... Kinematic internal variablen at tn
    #    CurState.Stress0      ... Stress point at unloading
    #
    # Output:
    #    NextStress            ... Stress at tn+1
    #    CurState.eP           ... Plastic strain at tn
    #    CurState.alphaISO     ... Isotropic internal variable at tn+1
    #    CurState.alphaKIN     ... Kinematic internal variablen at tn+1
    #    Cep                   ... Consistent tangent modulus

    #Static Parameters
    G = StaticParam.G
    K = StaticParam.K
    mm = StaticParam.mm
    hh = StaticParam.hh
    beta = StaticParam.beta
    RR = StaticParam.RR

    tol_rel = 1.0e-10
    meye = np.eye(6,1); meye[1]=1; meye[2]=1 
    small = 1.0e-10
    debugFlag = 1 #Set 0 for true, 1 for false

    Ce = GetCe(StaticParam)
    NextCep = Ce

    dev_dStrain = dev(dStrain)
    vol_dStrain = trace(dStrain)
    res = np.zeros((2,1))
    
    
    NextStress0 = CurState.Stress0
    dev_NextStress0 = dev(NextStress0)
    dev_CurStress = dev(CurStress)

    norm_dev_CurStress = normS(dev_CurStress)
    norm_dev_NextStress0 = normS(dev_NextStress0)

    CurKappa = CurState.Kappa
    CurPsi = CurState.Psi
    #Assume next Kappa and Psi are current (will overwrite for loading)
    NextKappa = CurKappa
    NextPsi = CurPsi
    NextalphaISO = CurState.alphaISO
    
    Iter_Psi = 0
    
    numerator = innerProduct((-(1 + CurKappa)*dev_CurStress - CurKappa*(1 + CurKappa)*(dev_CurStress - dev_NextStress0)), dev_dStrain, 3)
    denominator = innerProduct(( (1 + CurKappa)*dev_CurStress - CurKappa*dev_NextStress0), (dev_CurStress - dev_NextStress0), 1)

    if np.absolute(denominator) < small:
        loadingCond = 0
    else:
        loadingCond = numerator/denominator


    if loadingCond > 0.0:
        if debugFlag == 0:
            print('Unloading Happened')

        NextStress0 = CurStress
        dev_NextStress0 = dev(NextStress0)
        loadingCond = 0

    if loadingCond == 0:
        # ====================================================================================================
        # unloading (or beginning of loading) just happened
        if debugFlag == 0:
            print('Initial Loading') 
        
        #Initial pseudo-elastic Psi and Kappa at unloading point
        NextPsi = 2.0*G 
        NextKappa = 1.0e6
        dev_dStrainNorm = normE(dev_dStrain)
             
        if np.absolute(dev_dStrainNorm) < small:
            NextStress = CurStress
        else:
            #dStrain deviatoric unit vector
            dev_dStrainDir = dev_dStrain/dev_dStrainNorm
            NextH = evalH(StaticParam, NextKappa)
            res[0] = NextPsi*(1.0 + 3.0*G*beta/NextH)/(2.0*G)-1.0 
            res[1] = vectorNorm((dev_CurStress+(1.0+NextKappa)*NextPsi*convert2StressLike(dev_dStrain)), 1)/RR - 1.0
            res_norm = np.sqrt(res[0]**2 + res[1]**2)

            #Initialize Newton variables
            iter_counter = 0
            iter_max = 50
            tol_mat = tol_rel*res_norm
            incVar = np.zeros((2,1))
            Iter_Psi = 0
            for iter_counter in range(1, iter_max+1):
                if debugFlag == 0: 
                    print('Iteration = ', num2str(iter_counter), '  Norm = ', num2str(res_norm))
                if res_norm < (tol_mat + small):
                    NextStress = CurStress + K*vol_dStrain*meye + NextPsi*convert2StressLike(dev_dStrain)
                    break 

                temp = (dev_CurStress + (1 + NextKappa)*NextPsi*convert2StressLike(dev_dStrain))
                temp = temp / vectorNorm(temp,1)
                Ktan = np.zeros((2,2))
                Ktan[0,0] = (1.0 + 3.0*G*beta/NextH) / (2.0*G)
                Ktan[0,1] = (-3.0*G*NextPsi*beta*mm/hh/(NextKappa**(mm+1.0))/(2.0*G))
                Ktan[1,0] = ((1.0+NextKappa) * innerProduct(temp, dev_dStrain, 3)) / RR
                Ktan[1,1] = (innerProduct(temp, NextPsi*convert2StressLike(dev_dStrain), 1)) / RR

                incVar = np.linalg.solve(Ktan, res)  #\ operator is left matrix division, minimizes AX - B

                NextPsi = NextPsi - incVar[0]
                NextKappa = NextKappa - incVar[1]

                NextH = evalH(StaticParam, NextKappa)

                #Calculate New Residual
                res[0] = NextPsi*(1.0 + 3.0*G*beta/NextH)/(2.0*G)-1.0 
                res[1] = vectorNorm(dev_CurStress + (1.0 + NextKappa)*NextPsi*convert2StressLike(dev_dStrain), 1)/RR - 1.0
                res_norm = np.sqrt(res[0]**2 + res[1]**2)
                Iter_Psi = iter_counter

    else:

        # ====================================================================================================
        # Continuing Loading
        if debugFlag == 0: 
            print('Loading Continues')  
        
        
        CurH = evalH(StaticParam, CurKappa)
        #Pseudo-elastic assumption
        NextH = evalH(StaticParam, NextKappa)
        res[0] = NextPsi*(1.0 + 3.0*G*((1-beta)/CurH + beta/NextH))/(2.0*G)-1.0
        res[1] = vectorNorm(dev_CurStress + (1.0 + NextKappa)*NextPsi*convert2StressLike(dev_dStrain) + NextKappa*(dev_CurStress - dev_NextStress0), 1)/RR - 1.0
        res_norm = np.sqrt(res[0]**2+res[1]**2)

        #Initialize Newton variables
        iter_counter = 0
        iter_max = 50
        tol_mat = tol_rel*res_norm
        incVar = np.zeros((2,1))
        Iter_Psi = 0
        for iter_counter in range(1, iter_max+1): 
            if debugFlag == 0: 
                print('Iteration = ', num2str(iter_counter), '  Norm = ', num2str(res_norm))
            if res_norm < (tol_mat+small):
                NextStress = CurStress + K*vol_dStrain*meye + NextPsi*convert2StressLike(dev_dStrain)
                break 

            temp = (dev_CurStress + (1+NextKappa)*NextPsi*convert2StressLike(dev_dStrain) + NextKappa*(dev_CurStress - dev_NextStress0)); 
            temp = temp / vectorNorm(temp,1)
            Ktan = np.zeros((2,2))
            Ktan[0,0] = (1.0+3.0*G*((1-beta)/CurH + beta/NextH)) / (2.0*G)
            Ktan[0,1] = (-3.0*G*NextPsi*beta*mm/hh/(NextKappa**(mm+1.0))/(2.0*G))
            Ktan[1,0] = ((1.0+NextKappa) * innerProduct(temp, dev_dStrain, 3)) / RR
            Ktan[1,1] = (innerProduct(temp, dev_CurStress + NextPsi*convert2StressLike(dev_dStrain)-dev_NextStress0, 1)) / RR

            incVar = np.linalg.solve(Ktan,res) #Since Ktan is nxn: X = A\B | AX = B | X minimizes norm(AX - B)
            
            #Real next Psi, Kappa, and H
            NextPsi = NextPsi - incVar[0]
            NextKappa = NextKappa - incVar[1]
            NextH = evalH(StaticParam, NextKappa)

            #Calculate New Residual            
            res[0] = NextPsi*(1.0+3.0*G*((1-beta)/CurH + beta/NextH))/(2.0*G) - 1.0
            res[1] = vectorNorm(dev_CurStress+(1.0+NextKappa)*NextPsi*convert2StressLike(dev_dStrain)+ NextKappa*(dev_CurStress - dev_NextStress0), 1)/RR - 1.0
            res_norm = np.sqrt(res[0]**2 + res[1]**2)
            
            Iter_Psi = iter_counter


    NextStress = CurStress + K*vol_dStrain*meye + NextPsi*convert2StressLike(dev_dStrain)

    # Update State
    state = namedtuple('state', ['eP','alphaISO','Stress0', 'Kappa', 'Psi', 'Iter_Psi'])
    NextState = state(0, NextalphaISO, NextStress0, NextKappa, NextPsi, Iter_Psi)
    
    #Get 6x6 representations of Ivol and Idev
    Ivol = GetIvol()
    Idev = GetIdev()
    #Ivol = np.array([[np.ones((3,3)), np.zeros((3,3))], [np.zeros((3,3)), np.zeros((3,3))]]).reshape(6,6) # 3Ivol
    #NOTE: NextCep is equivalent to 3KIvol + 2GIdev when NextPsi is at its elastic state: 2G
    NextCep = 3*K*Ivol + NextPsi*Idev
    
    return NextStress, NextState, NextCep

In [80]:
def BSDriver(LoadCase):
    # BoundingSurface J2 with kinematic hardening 
    # Written by Pedro Arduino, Mar. 22 2019
    # Copyright Arduino Computational Geomechanics Group
    # Ported into Python/Jupyter Notebook by Justin Bonus, Jul. 2019
    #
    #
    # LoadCase:
    #    1 ... proportionally increasing strain
    #    2 ... cyclic strain
    #    3 ... proportionally increasing stress
    #    4 ... cyclic stress
    #
    # ======  LOADING CASES ==================================================
    
    import numpy as np
    from collections import namedtuple
    
    nPoints = n
    loadFactor = f
    ## Switch for LoadCases:
    ## Pseudo-switch created by using python dictionary to hold LoadCase functions
    def case_one():
        case_one.time   = np.linspace(0,1,nPoints+1)
        case_one.strain = np.array([ 0.05, -0.015, -0.015, 0.000, 0.000, 0.000 ]).reshape(6,1) * loadFactor * case_one.time
        case_one.StressDriven = 0
        return case_one
    def case_two():
        nCycles = 3
        omega   = 0.15
        case_two.time   = np.linspace(0,nCycles*2*np.pi/omega,nCycles*nPoints+1);
        case_two.strain = np.array([ 0.00, -0.000, -0.000, 0.050, 0.000, 0.000 ]).reshape(6,1) * f* np.sin( omega*case_two.time )      
        case_two.StressDriven = 0 
        return case_two
    def case_three():
        case_three.time   = np.linspace(0,1,nPoints+1)       
        case_three.stress = np.array([[0.100],
                           [0.000],
                           [0.000],
                           [0.000],
                           [0.000],
                           [0.000]])*f*case_three.time + 0.0*np.array([1,1,1,0,0,0]).reshape(6,1)*np.ones( case_three.time.shape )            
        case_three.StressDriven = 1    
        return case_three
    def case_four():
        nCycles = 3
        omega   = 0.15
        case_four.time   = np.linspace(0, nCycles*2*np.pi/omega, nCycles*nPoints+1)
        case_four.stress = np.array([[0.000],
                           [0.000],
                           [0.000], #.01, .03, -.01, .05, 0, -.02
                           [0.045],
                           [0.000],
                           [0.000]])*f*np.sin( omega*case_four.time ) + 0.0*np.array([1,1,1,0,0,0]).reshape(6,1)*np.ones( case_four.time.shape )            
        case_four.StressDriven = 1          
        return case_four

    case_switcher = {
        1: case_one,
        2: case_two,
        3: case_three,
        4: case_four
    }    

    case = case_switcher.get(LoadCase, lambda: "Invalid LoadCase")
    case() #Runs the LoadCase function. Creates: case.time, case.strain | case.stress, case.StressDriven
    time, StressDriven = case.time, case.StressDriven 
    if StressDriven:
        stress = case.stress
        strain = np.zeros((6,1)) #initialize empty 6x1 strain numpy array for stress-driven scenario
    else:
        strain = case.strain
        stress = np.zeros((6,1)) #initialize empty 6x1 stress numpy array for strain-driven scenario
    
    Stress0 = np.zeros((6,1)) #Initialize first 'unloading' point
    StrainDriven = int(not StressDriven)

    # ========================================================================
    # ---- MATERIAL PARAMETERS
    # Static Parameters

    # Static Parameters
    E = 20 #Elastic Modulus  MPa
    v= 0.49 #Poissons ratio, less than 0.5 to allow compresibility
    G = E/(2*(1+v)) #Shear modulus
    K = E/(3*(1-2*v)) #Bulk modulus
    Kmod = 0 #Isotropic Hardening
    Su = 0.061 #Yield stress in 1-D tension test MPa
    hh = G #kinematic hardening parameter
    mm = 1.0 #kinematic hardening parameter
    beta = 0.5 #midpoint integration
    RR = np.sqrt(8/3)*Su

    #namedtuple used to organzie related variables, similar to a structure
    static = namedtuple('StaticParam',['E','v','G','K','Kmod','Su','hh','mm','beta','RR'])
    StaticParam = static(E,v,G,K,Kmod,Su,hh,mm,beta,RR)


    # ========================================================================
    # ---- INITIAL CONDITIONS

    # Initialize the state variables
    if StrainDriven:
        IniStress = -0.0*(np.array([1, 1, 1, 0, 0, 0]).reshape(6,1))
        IniStrain = np.linalg.solve(GetCe(StaticParam), IniStress) #Check if GetCe compacts to nxn
    elif StressDriven:
        IniStress =  0.0*(np.array([1, 1, 1, 0, 0, 0]).reshape(6,1))
        IniStrain =  0.0*(np.array([1, 1, 1, 0, 0, 0]).reshape(6,1))   

    #Structure for IniState (initial state parameters, static) and CurState (changing state parameters)
    state = namedtuple('state', ['eP','alphaISO','Stress0', 'Kappa', 'Psi', 'Iter_Psi'])
 
    eP = 0.0*(np.array([1, 1, 1, 0, 0, 0]).reshape(6,1))
    alphaISO = 0.0  
    Stress0 = 0.0*(np.array([1, 1, 1, 0, 0, 0]).reshape(6,1))
    Kappa = 0.0
    Psi = 0.0
    Iter_Psi = 0
    IniState = state(eP, alphaISO, Stress0, Kappa, Psi, Iter_Psi)

    # For first iteration
    CurStress = IniStress
    CurStrain = IniStrain
    CurState  = IniState

    # Variables used for plotting
    alphaISO_plot, j2_plot, j2e_plot, stress_var_plot, stress_var2_plot = [], [], [], [], [] #Initiliaze list format
    alphaISO_plot.append(0) #Python list allows for easy data addition
    strain[:,0] = CurStrain.T - IniStrain.T 
    stress[:,0] = CurStress.T
    j2_plot.append(0)
    j2e_plot.append(0)
    stress_var_plot.append(0)
    Stress0[:,0] = CurStress.T
    Iter = np.zeros(time.shape)
    Iter_Inner = np.zeros(time.shape)    

    # ========================================================================
    # ---- COMPUTATION CYCLES

    if StrainDriven:
        #StrainDriven
        for i in range(1, (len(strain[0]) )):

            NextStrain = strain[:,i] + IniStrain.T
            dStrain = strain[:,i] - strain[:, i-1] #Driving variable
            
            #Current BSRadialMap is a function, will be transformed into a class eventually
            NextStress, NextState, NextCep = BSRadialMap(dStrain, StaticParam, CurStress, CurState)

            # Update Stress, Strain, and State
            CurStress = NextStress
            CurState = NextState
            
            # Variables created for plotting purposes
            alphaISO_plot.append(CurState.alphaISO)
            stress = np.append(stress, CurStress, 1)
            j2_plot.append(GetJ2(CurStress))
            stress_var_plot.append(np.sqrt(2*j2_plot[i])*np.sqrt(3/2)*np.sign(stress[0,i] - stress[1,i]))
            stress_var2_plot.append((stress[0,i] - stress[1,i]))
            Stress0 = np.append(Stress0, CurState.Stress0, 1)
            Iter_Inner[i] = CurState.Iter_Psi
            
    elif StressDriven:
        # StressDriven driver
        # set tolerance value for iterative procedure(s)
        TOLERANCE = 1e-10 

        for i in range(0, len(stress[0])-1):

            # initialize strain epsilon_{n+1}^{(0)} = eps_{n} using the old state
            # (this is the initial approximation for eps_{n+1}
            if i == 0:
                # special settings for initial values at t_1
                NextStrain = np.array([0,0,0,0,0,0]).reshape(6,1)
                dStrain = np.array([0,0,0,0,0,0]).reshape(6,1)
                CurState = IniState
            else:
                NextStrain = CurStrain
                dStrain = np.array([0,0,0,0,0,0]).reshape(6,1)

            NextStress, NextState, Cep = BSRadialMap(dStrain, StaticParam, CurStress, CurState)

            RR = stress[:, i].reshape(6,1) - NextStress
            RR = RR.reshape(6,1)
            RR0 = normS(RR)

            # reset iteration counter
            kk = 0
            # iterate until convergence
            while normS(RR)/RR0 > TOLERANCE:
                
                # update strain from eps_{n+1}^{(k)} to eps_{n+1}^{(k+1)}
                dStrain = np.linalg.solve(Cep, RR)
                NextStrain = NextStrain + dStrain

                # compute material response for estimated strain state
                # NOTE: the state variables are taken at t_n
                NextStress, NextState, Cep = BSRadialMap(dStrain, StaticParam, CurStress, CurState)
                #print('NextStress:',NextStress)
                #print('Stress0:',NextState.Stress0)
                # check for equilibrium
                RR = stress[:,i].reshape(6,1) - NextStress
                RR = RR.reshape(6,1)
                kk = kk + 1
                # emergence exit if procedure does not converge            
                if kk > 3:
                    kk = kk
                    #print('procedure slow to converge. Error : ', normS( RR )/RR0)
                
                if kk > 20:
                    print('procedure did not converge. Error : ', normS( RR )/RR0)
                    print('YOUR TANGENT Cep IS WRONG', normS( RR )/RR0)
                    break

                Iter[i] = kk
                CurStress = NextStress
                CurState = NextState


            # Update State variables for next step
            CurStress = NextStress
            CurStrain = NextStrain
            CurState  = NextState

            # Update variables for plotting purposes
            strain = np.append(strain, CurStrain, 1)
            alphaISO_plot.append(CurState.alphaISO)
            j2_plot.append(GetJ2(CurStress))
            stress_var_plot.append(np.sqrt(2*j2_plot[i])*np.sqrt(3/2)*np.sign(stress[3,i]))
            Stress0 = np.append(Stress0, CurState.Stress0, 1)
            Iter_Inner[i] = CurState.Iter_Psi
            
    DriverOutput = namedtuple('DriverOutput',['StaticParam','time','strain','stress','alphaISO','j2','stress_var','stress_var2', 'Stress0','Iter', 'Iter_Inner'])
    DriverOutput = DriverOutput(StaticParam, time, strain, stress, alphaISO_plot, j2_plot, stress_var_plot, stress_var2_plot, Stress0, Iter, Iter_Inner)
    
    return DriverOutput
    
    # =========================================================================

# Solved

In [3]:
def solvedBSRadialMap(dStrain, StaticParam, CurStress, CurState):
    # Radial Return Algorithm for J2 Bounding Surface Plasticity Model
    # Original model by Borjas & Amies 1994
    # Base radialmap by Pedro Arduino, Mar. 22 2019
    # Copyright Arduino Computational Geomechanics Group
    # Precomputed matrix enabled version by Justin Bonus, Jul. 2019
    #
    #
    # Input:
    #    dStrain               ... Strain differential, tn to tn+1
    #    StaticParam.E         ... Young's modulus
    #    StaticParam.v         ... poissons ratio
    #    StaticParam.G         ... Shear modulus
    #    StaticParam.K         ... Bulk modulus
    #    StaticParam.hh        ... Kinematic Hardening parameter
    #    StaticParam.mm        ... Kinematic hardening parameter
    #    StaticParam.beta      ... Integration parameter (0=expl, 1=impl)
    #    StaticParam. RR       ... Bounding surface radius
    #    CurStress             ... Stress at tn
    #    CurState.eP           ... Plastic strain at tn
    #    CurState.alphaISO     ... Isotropic internal variable at tn
    ##   CurState.alphaKIN     ... Kinematic internal variablen at tn
    #    CurState.Stress0      ... Stress point at unloading
    #
    # Output:
    #    NextStress            ... Stress at tn+1
    #    CurState.eP           ... Plastic strain at tn
    #    CurState.alphaISO     ... Isotropic internal variable at tn+1
    #    CurState.alphaKIN     ... Kinematic internal variablen at tn+1
    #    Cep                   ... Consistent tangent modulus

    #Static Parameters
    G = StaticParam.G
    K = StaticParam.K
    mm = StaticParam.mm
    hh = StaticParam.hh
    beta = StaticParam.beta
    ER = StaticParam.RR

    meye = np.eye(6,1); meye[1]=1; meye[2]=1 
    small = 1.0e-10 # Tolerance for small dStrain steps, AKA no stress change
    debugFlag = 0 #Set 0 for true, 1 for false

    #=================================
    # -----INDEX MAPPING AND ROTATIONS

    # Change these to passed values
    #res_x = 
    #res_y = 
    cen_x = int(np.floor(res_x/2))
    cen_y = int(np.floor(res_y/2))
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    Ce = GetCe(StaticParam)
    NextCep = Ce
    
    dev_dStrain = dev(dStrain)
    vol_dStrain = trace(dStrain)
    con_dStrain = convert2StressLike(dStrain)
    #dStrainP = principal(dStrain)
    dev_con_dStrain = dev(con_dStrain)
    vol_con_dStrain = trace(dStrain)
    pDev_con_dStrain = principal(dev_con_dStrain)

    
    
    NextStress0 = CurState.Stress0 
    dev_NextStress0 = dev(NextStress0)
    dev_CurStress = dev(CurStress)

    NextStress0P = principal(NextStress0)
    pDev_NextStress0 = principal(dev_NextStress0)
    CurStressP = principal(CurStress)
    pDev_CurStress = principal(dev_CurStress)
    
    
    CurKappa = CurState.Kappa
    CurPsi = CurState.Psi
    psi = CurState.Psi
    NextKappa = CurKappa # Temporary
    NextH = CurState.H
    NextPsi = CurPsi # Temporary
    NextalphaISO = 0
    Iter_H = 0
    Iter_Psi = 0
    
    
    # Unit-vectors
    north = np.array([[np.sqrt(2/3)],[-np.sqrt(1/6)],[-np.sqrt(1/6)]]) #\Pi-plane north unit-vector
    east = np.array([[0],[np.sqrt(1/2)],[-np.sqrt(1/2)]]) #\Pi-plane east unit-vector
    south = -north

    #=================================
    # -----UNLOADING CHECK
    numerator = innerProduct((-(1 + CurKappa)*dev_CurStress - CurKappa*(1 + CurKappa)*(dev_CurStress - dev_NextStress0)), dev_dStrain, 3)
    denominator = innerProduct(( (1 + CurKappa)*dev_CurStress - CurKappa*dev_NextStress0), (dev_CurStress - dev_NextStress0), 1)

    if np.absolute(denominator) < small:
        loadingCond = 0
    else:
        loadingCond = numerator/denominator

    if loadingCond > 0.0:
        if debugFlag == 0:
            print('Unloading Happened')
        NextStress0 = CurStress
        dev_NextStress0 = dev(NextStress0)
        NextStress0P = principal(NextStress0)
        pDev_NextStress0 = principal(dev_NextStress0)
        CurKappa = 10e6
        NextPsi = 2*G
        Unloading = True
        loadingCond = 0
    else:
        loadingCond = 0
        Unloading = False

    #Unloading 
    #=================================

    if loadingCond == 0:
        
        dev_dStrainNorm = normE(dev_dStrain)
        if np.absolute(dev_dStrainNorm) < small:
            NextStress = CurStress

        else:

            #=========================
            # -----Rotate North & Map to Index Space     
            #indexNextStress0 = np.array([cen_x,]).reshape(2,1)
            indexNextStress0 = APiToOPi(stressToIndex(pDev_NextStress0, res_x, res_y, RR, sliced=False),res_x) # O\Pi Index float space
            indexCurStress = APiToOPi(stressToIndex(pDev_CurStress, res_x, res_y, RR, sliced=False),res_x) # O\Pi Index float space
            
            north = np.array([0,1]).reshape(2,1)
            east = np.array([1,0]).reshape(2,1)
            
            if np.all(indexNextStress0 == 0):
                northUnloadingAngle = 0. # If zero vector stress, set angles to 0
                eastUnloadingAngle = 0.
            else:
                northUnloadingAngle = angleIndex(north, indexNextStress0) # Angle of unloading stress from north
                #eastUnloadingAngle = angleIndex(east, indexNextStress0) # Angle of unloading stress from east
            #if eastUnloadingAngle > np.pi/2:
            #    northUnloadingAngle = -northUnloadingAngle # Check if west of centerline, adjust
            

            #indexNorthNextStress0 = OPiToAPi(indexRotator(indexNextStress0, -northUnloadingAngle),res_x)# A\Pi Index float space
            indexNorthNextStress0 = indexNextStress0
            indexNorthNextStress0[0] = 0; indexNorthNextStress0[1] = np.sqrt(indexNextStress0[0]**2 + indexNextStress0[1]**2)
            indexNorthNextStress0 = OPiToAPi(indexNorthNextStress0, res_x)
            indexNorthCurStress = OPiToAPi(indexRotator(indexCurStress, -northUnloadingAngle),res_x) # A\Pi Index float space         
            northDStrain = quatHydroRotator(pDev_con_dStrain, -northUnloadingAngle)  # dev_dStrain for South aligned CurStress, North aligned Stress0
            southDStrain = -northDStrain
            # the strain is not polarized currently, not good
            
            #=========================
            # -----Access \kappa Matrix
            # Find layer containing appropiate 1D North NextStress0 position
            if indexNorthNextStress0[1] == 0:
                indexNorthNextStress0[1] = 1
            if indexNorthNextStress0[1] > cen_y:
                print('Fix polarity of rotation')
            if indexNorthNextStress0[1] < 0:
                print('Center Error, negative')
                
            layerK = int(np.round(cen_y - indexNorthNextStress0[1])) # Layer cen_y is at BS, layer 0 at origin
            Iter_H = layerK
            jK = int(np.round(indexNorthCurStress[1]))
            iK = int(np.round(indexNorthCurStress[0]))
            cartK = kappaM[:, :, layerK] # Holds relevant \kappa matrix layer
            CurKappa = cartK[jK, iK] # Determine current H', could pass between time-step
            CurH = (hh * CurKappa)**mm

            # ========================        
            # -----Search for H' & \psi pair
            # Find paired psi and NextH values to satisfy formula 

            # Search line for \psi and H' pairs
            rr, cc = psiSearch(northDStrain, indexNorthCurStress, res_x, res_y, RR)   
            
            #rrN, ccN = psiSearch(northDStrain, indexNorthCurStress, res_x, res_y, RR)    
            #rrS, ccS = psiSearch(southDStrain, indexNorthCurStress, res_x, res_y, RR)
            #rr, cc = [],[]
            #end = min(len(rrS),len(rrN))
            #for i in range(min(len(rrS),len(rrN))):
            #    rr.append(rrS[i])
            #    rr.append(rrN[i])
            #for i in range(min(len(ccS),len(ccN))):
            #    cc.append(ccS[i])
            #    cc.append(ccN[i])
            #if len(rrN) > end:
            #    for e in range(end,len(rrN)):
            #        rr.append(rrN[e])
            #if len(rrS) > end:
            #    for e in range(end,len(rrS)):
            #        rr.append(rrS[e])
            #if len(ccN) > end:
            #    for e in range(end,len(ccN)):
            #        cc.append(ccN[e])
            #if len(ccS) > end:
            #    for e in range(end,len(ccS)):
            #        cc.append(ccS[e])
            #rr, cc = psiSearch(northDStrain, indexNorthCurStress, res_x, res_y, RR)  
            
            tol_R = 1e-1 # Tolerance for equation 37, major speed increase when looser
            hold_R = 10e6 # Initialize tolerance comparison eq. 37
            Iter_Psi = 0 # Reset psi-search iterations
            #Iter_H = 0 # Redundant, currently used for tracking random  values of interest

            # 'stop' is a very important parameter
            # High values will slow the program/increase chance of error
            # Low values will limit the stress movement/increase error
            # Will eventually become 'predictively' set
            stop = int(res_x/4) # How many points to evaluate on search line (more needed for larger steps/finer resolution)
            stop = np.linspace(0, stop-1, stop)
            hold = None
            for j, i, s in zip(rr, cc, stop):
                Iter_Psi = s # Iteration of search function
                NextKappa = float(cartK[j,i]) # H' value of NextStress cell
                if NextKappa == 0:
                    # Skip if cell is on/outside of bounding surface
                    continue   
                if NextKappa > CurState.Kappa and Unloading == False:
                    continue
                NextH = float((NextKappa*hh)**(mm)) # Compute hardening moduli
                psi = (2*G) / (1 + 3*G*((1-beta)/NextH + beta/NextH)) # \psi value of specific \psi-H' pair
                if psi > CurState.Psi and Unloading == False:
                    continue
                # Solve base equation, zero res_R means pair matches true function (i.e. no error)
                res_R = np.abs(vectorNorm(dev_CurStress + psi*convert2StressLike(dev_dStrain) + NextKappa*(dev_CurStress + psi*convert2StressLike(dev_dStrain) - dev_NextStress0),1)-RR)

                # If in an acceptable range of true solution, break loop to save time
                if res_R <= tol_R:
                    hold = False
                    break

                #Hold best solution if one below tolerance isn't found
                if res_R < hold_R:
                    hold_NextKappa = NextKappa
                    hold_NextH = NextH
                    hold_psi = psi
                    hold_Iter_Psi = Iter_Psi
                    hold_R = res_R
                    hold = True

            # If tolerances were not met, give the best pair found
            if hold == True:
                NextKappa = hold_NextKappa 
                NextH = hold_NextH
                psi = hold_psi
                Iter_Psi = hold_Iter_Psi
                res_R = hold_R

        #==================================
        # -----Solve Constitutive Equation
        NextStress = CurStress + K*(vol_dStrain)*meye + psi*convert2StressLike(dev_dStrain)
        #NextStress = CurStress + K*vol_dStrain*meye + (1/2)*psi*dev_dStrain

        # Update State
        state = namedtuple('state', ['eP','alphaISO', 'Stress0', 'Iter_H', 'Iter_Psi', 'Kappa', 'H', 'Psi'])
        NextState = state(0, NextalphaISO, NextStress0, Iter_H, Iter_Psi, NextKappa, NextH, psi)

        Ivol = GetIvol()
        Idev = GetIdev()

        NextCep = 3*K*Ivol + psi*Idev
    
    return NextStress, NextState, NextCep

In [73]:
def solvedBSDriver(LoadCase):
    # Precomputed Matrix Enabled BoundingSurface J2 with kinematic hardening 
    # Base driver by Pedro Arduino, Mar. 22 2019
    # Copyright Arduino Computational Geomechanics Group
    # Precomputed solution matrice version by Justin Bonus, Aug. 2019
    #
    #
    # LoadCase:
    #    1 ... proportionally increasing strain
    #    2 ... cyclic strain
    #    3 ... proportionally increasing stress
    #    4 ... cyclic stress
    #
    # ======  LOADING CASES ==================================================
    
    import numpy as np
    from collections import namedtuple
    from skimage.draw import line
    
    nPoints = n

    ## Switch for LoadCases:
    ## Pseudo-switch created by using python dictionary to hold LoadCase functions
    def case_one():
        # Monotonic strain
        case_one.time   = np.linspace(0,1,nPoints+1)
        case_one.strain = np.array([ 0.05, -0.015, -0.015, 0.000, 0.000, 0.000 ]).reshape(6,1) * case_one.time
        case_one.StressDriven = 0
        return case_one
    
    def case_two():
        # Cyclic strain
        nCycles = 3
        omega = 0.15
        case_two.time = np.linspace(0,nCycles*2*np.pi/omega,nCycles*nPoints+1)
        case_two.strain = np.array([ 0.00, -0.000, -0.000, 0.03, 0.000, 0.000 ]).reshape(6,1) * np.sin( omega*case_two.time )      
        case_two.StressDriven = 0
        return case_two
    
    def case_three():
        # Monotonic stress
        case_three.time   = np.linspace(0,1,nPoints+1)
        case_three.stress = np.array([[0.100],
                           [0.000],
                           [0.000],
                           [0.000],
                           [0.000],
                           [0.000]])*case_three.time + 0.0*np.array([1,1,1,0,0,0]).reshape(6,1)*np.ones( case_three.time.shape )            
        case_three.StressDriven = 1
        return case_three
    
    def case_four():
        # Cyclic stress
        nCycles = 3
        omega   = 0.15
        case_four.time   = np.linspace(0, nCycles*2*np.pi/omega, nCycles*nPoints+1)
        case_four.stress = np.array([[0.000],
                           [0.000],
                           [0.000], 
                           [0.050],
                           [0.000],
                           [0.000]])*np.sin( omega*case_four.time ) + 0.0*np.array([1,1,1,0,0,0]).reshape(6,1)*np.ones( case_four.time.shape )            
        case_four.StressDriven = 1         
        return case_four
    
    def case_five():
        # Spiral strain
        Su = 0.061 #Undrained shear strength, MPa
        R = Su*(8/3)**0.5
        nRevs = 10/3
        nCycles = 3
        omega   = 0.15
        case_five.time = np.linspace(0, nCycles*2*np.pi/omega, nCycles*nPoints)

        # Spiral
        p = np.linspace(0,nRevs*2*np.pi,nCycles*nPoints)
        a = 3*R
        k = R*np.sqrt(2)/2
        x=a*np.e**(k*p)*np.sin(p)
        y=a*np.e**(k*p)*np.cos(p)
        y = y-a; y = -y # Center and flip

        case_five.strain = np.array([[np.sqrt(2/3)*0.01],[0],[0]])*y + np.array([[0],[(1/2)*0.01],[-(1/2)*0.01]])*x
        case_five.strain = np.append(case_five.strain, np.flip(case_five.strain,1),1).reshape(3,2*nCycles*nPoints)
        case_five.time = np.append(case_five.time,case_five.time[::-1])
        case_five.StressDriven = 0
        return case_five
    
    def case_six():
        # Spiral Stress
        Su = 0.061 #Undrained shear strength, MPa
        R = Su*(8/3)**0.5
        nRevs = 10/3
        nCycles = 3
        omega   = 0.15
        case_six.time = np.linspace(0, nCycles*2*np.pi/omega, nCycles*nPoints)

        # Spiral
        p = np.linspace(0,nRevs*2*np.pi,nCycles*nPoints)
        a = 3*R
        k = R*np.sqrt(2)/2
        x=a*np.e**(k*p)*np.sin(p)
        y=a*np.e**(k*p)*np.cos(p)
        y = y-a; y = -y # Center and flip

        case_six.stress = np.array([[np.sqrt(2/3)*0.05],[0],[0]])*y + np.array([[0],[(1/2)*0.05],[-(1/2)*0.05]])*x
        case_six.stress = np.append(case_six.stress, np.flip(case_six.stress,1),1).reshape(3,2*nCycles*nPoints)
        case_six.time = np.append(case_six.time,case_six.time[::-1])
        case_six.StressDriven = 1
        return case_six
    
    case_switcher = {
        1: case_one,
        2: case_two,
        3: case_three,
        4: case_four,
        5: case_five,
        6: case_six
    }    

    case = case_switcher.get(LoadCase, lambda: "Invalid LoadCase")
    case() #Runs the LoadCase function. Creates: case.time, case.strain | case.stress, case.StressDriven
    time, StressDriven = case.time, case.StressDriven 
    if StressDriven == 1:
        stress = case.stress
        strain = np.zeros((6,1)) #initialize empty 3x1 strain numpy array for stress-driven scenario
    else:
        strain = case.strain
        stress = np.zeros((6,1)) #initialize empty 3x1 stress numpy array for strain-driven scenario
    Stress0 = np.zeros((6,1)) #Initialize first 'unloading' point
    
    # ========================================================================
    # ---- MATERIAL PARAMETERS
    # Static Parameters
    E = 20 #Elastic Modulus  MPa
    v= 0.49 #Poissons ratio, less than 0.5 to allow compresibility
    G = E/(2*(1+v)) #Shear modulus
    K = E/(3*(1-2*v)) #Bulk modulus
    Kmod = 0 #Isotropic Hardening
    Su = 0.061 #Yield stress in 1-D tension test MPa
    hh = G #kinematic hardening parameter
    mm = 1.0 #kinematic hardening parameter
    beta = 0.5 #midpoint integration
    RR = np.sqrt(8/3)*Su # Make RR

    # Class structure
    static = namedtuple('StaticParam',['E','v','G','K','Kmod','Su','hh','mm','beta','RR'])
    StaticParam = static(E,v,G,K,Kmod,Su,hh,mm,beta,RR)

    # ========================================================================
    # ---- LOAD PRECOMPUTED MATRICES
    #res_x = 
    #res_y = 
    # Currently matrices are loaded in the notebook before reaching the driver,
    # it's unnecesary to reload them.
    # Read \kappa Matrix
    #fn = 'kappaMatrix_' + str(res_x) + '_' + str(res_y) + '.txt'
    #kappaM = np.loadtxt(fn).reshape((res_y,res_x,int(np.floor(res_y/2))+1)) # Returns 3D array

    # ========================================================================
    # ---- INITIAL CONDITIONS

    # Initialize the state variables
    if StressDriven == True:
        IniStress = 0.0*np.array([1, 1, 1, 0, 0, 0]).reshape(6,1)
        IniStrain = np.linalg.solve(GetCe(StaticParam), IniStress) #Check if GetCe compacts to nxn
    else:
        IniStress =  0.0*np.array([1, 1, 1, 0, 0, 0]).reshape(6,1)
        IniStrain =  0.0*np.array([1, 1, 1, 0, 0, 0]).reshape(6,1)   

    eP = 0.0*(np.array([1, 1, 1, 0, 0, 0]).reshape(6,1))
    alphaISO = 0.0  
    Stress0 = 0.0*(np.array([1, 1, 1, 0, 0, 0]).reshape(6,1))
    Kappa = 1e6 #maxKappa
    H = StaticParam.hh*Kappa**StaticParam.mm
    Psi = 2*StaticParam.G
    
    # Structure for IniState (initial state parameters, static) and CurState (changing state parameters)
    state = namedtuple('state', ['eP','alphaISO','Stress0','Kappa','H','Psi'])
    IniState = state(eP, alphaISO, Stress0, Kappa, H, Psi)

    # For first iteration
    CurStress = IniStress
    CurStrain = IniStrain
    CurState  = IniState

    # Variables used for plotting
    kappa, H, psi = [],[],[]
    strain[:,0] = CurStrain.T - IniStrain.T 
    stress[:,0] = CurStress.T - IniStress.T #Include -ini ?
    Stress0[:,0] = CurStress.T
    alphaISO_plot, j2_plot, j2e_plot, stress_var_plot, stress_var2_plot = [],[],[],[],[] #Initiliaze list format
    alphaISO_plot.append(0) #Python list appending for easy data addition, could be faster if array size was initialized entirely
    j2_plot.append(0)
    j2e_plot.append(0)
    stress_var_plot.append(0)
    Iter_H = np.zeros(time.shape)
    Iter_Psi = np.zeros(time.shape)
    Iter = np.zeros(time.shape)
    maxKappa = 10e5 # Maximum considered \kappa value, stand in for infinity
    maxH = StaticParam.hh*maxKappa**StaticParam.mm # Hardening function dependent
    maxPsi = 2*StaticParam.G
    kappa.append(maxKappa) # Assumed to start at unloading point 
    H.append(maxH) # Assumed to start at unloading point
    psi.append(maxPsi) # Assumed to start at unloading point


    # ========================================================================
    # ---- COMPUTATION CYCLES

    if StressDriven == False:
        # Strain-driven
        for i in range(1, (len(strain[0]) )):
            
            # Input strain
            NextStrain = strain[:,i] + IniStrain.T
            dStrain = (strain[:,i] - strain[:, i-1]).reshape(6,1) #Driving variable
            NextStress, NextState, NextCep = solvedBSRadialMap(dStrain, StaticParam, CurStress, CurState)

            # Update Stress, Strain, and State
            CurStress = NextStress
            CurState = NextState
            
            # Variables created for plotting purposes
            stress = np.append(stress, CurStress, 1)
            pseudoStress = stress
            Stress0 = np.append(Stress0, CurState.Stress0, 1)
            alphaISO_plot.append(CurState.alphaISO)
            j2_plot.append(0)
            stress_var_plot.append(np.sqrt(2*j2_plot[i])*np.sqrt(3/2)*np.sign(stress[0,i] - stress[1,i]))
            stress_var2_plot.append((stress[0,i] - stress[1,i]))
            Iter_H[i] = CurState.Iter_H
            Iter_Psi[i] = CurState.Iter_Psi
            kappa.append(CurState.Kappa)
            H.append(CurState.H)
            psi.append(CurState.Psi)
            
    elif StressDriven == True:          
        # StressDriven driver
        # set tolerance value for iterative procedure(s)
        TOLERANCE = 1e-10 
        pseudoStress = IniStress
        for i in range(0, len(stress[0])-1):

            # initialize strain epsilon_{n+1}^{(0)} = eps_{n} using the old state
            # (this is the initial approximation for eps_{n+1}
            if i == 0:
                # special settings for initial values at t_1
                NextStrain = np.array([0,0,0,0,0,0]).reshape(6,1)
                dStrain = np.array([0,0,0,0,0,0]).reshape(6,1)
                CurState = IniState
            else:
                NextStrain = CurStrain
                dStrain = np.array([0,0,0,0,0,0]).reshape(6,1)

            NextStress, NextState, Cep = solvedBSRadialMap(dStrain, StaticParam, CurStress, CurState)

            RR = stress[:, i].reshape(6,1) - NextStress
            RR = RR.reshape(6,1)
            RR0 = normS(RR)

            # reset iteration counter
            kk = 0
            # iterate until convergence
 
            while normS(RR)/RR0 > TOLERANCE:
                
                # update strain from eps_{n+1}^{(k)} to eps_{n+1}^{(k+1)}
                dStrain = np.linalg.solve(Cep, RR)
                NextStrain = NextStrain + dStrain

                # compute material response for estimated strain state
                # NOTE: the state variables are taken at t_n
                NextStress, NextState, Cep = solvedBSRadialMap(dStrain, StaticParam, CurStress, CurState)

                # check for equilibrium
                RR = stress[:,i].reshape(6,1) - NextStress
                RR = RR.reshape(6,1)
                kk = kk + 1
                # emergence exit if procedure does not converge            
                if kk > 3:
                    kk = kk
                    #print('procedure slow to converge. Error : ', normS( RR )/RR0)
                
                if kk > 20:
                    #print('procedure did not converge. Error : ', normS( RR )/RR0)
                    #print('YOUR TANGENT Cep IS WRONG', normS( RR )/RR0)
                    break

                Iter[i] = kk
                CurStress = NextStress
                CurState = NextState
                


            # Update State variables for next step
            CurStress = NextStress
            CurStrain = NextStrain
            CurState  = NextState
            
            # Variables created for plotting purposes
            strain = np.append(strain, CurStrain, 1)
            pseudoStress = np.append(pseudoStress, CurStress, axis=1)
            Stress0 = np.append(Stress0, CurState.Stress0, 1)
            alphaISO_plot.append(CurState.alphaISO)
            j2_plot.append(0)
            stress_var_plot.append(np.sqrt(2*j2_plot[i])*np.sqrt(3/2)*np.sign(stress[0,i] - stress[1,i]))
            stress_var2_plot.append((stress[0,i] - stress[1,i]))
            Iter_H[i] = CurState.Iter_H
            Iter_Psi[i] = CurState.Iter_Psi
            kappa.append(CurState.Kappa)
            H.append(CurState.H)
            psi.append(CurState.Psi)
            
    DriverOutput = namedtuple('DriverOutput',['StaticParam','time','strain','stress','pseudoStress','alphaISO','j2','stress_var','stress_var2', 'Stress0', 'Iter_H', 'Iter_Psi', 'kappa', 'H', 'psi'])
    DriverOutput = DriverOutput(StaticParam, time, strain, stress, pseudoStress, alphaISO_plot, j2_plot, stress_var_plot, stress_var2_plot, Stress0, Iter_H, Iter_Psi, kappa, H, psi)

    return DriverOutput
    # =========================================================================

# Psi Version



In [2]:
def psiBSRadialMap(dStrain, StaticParam, CurStress, CurState, kappaMatrix):
    # Radial Return Algorithm for J2 Bounding Surface Plasticity Model
    # Original model by Borjas & Amies 1994
    # Base radialmap by Pedro Arduino, Mar. 22 2019
    # Copyright Arduino Computational Geomechanics Group
    # Precomputed matrix enabled version by Justin Bonus, Jul. 2019
    #
    #
    # Input:
    #    dStrain               ... Strain differential, tn to tn+1
    #    StaticParam.E         ... Young's modulus
    #    StaticParam.v         ... poissons ratio
    #    StaticParam.G         ... Shear modulus
    #    StaticParam.K         ... Bulk modulus
    #    StaticParam.hh        ... Kinematic Hardening parameter
    #    StaticParam.mm        ... Kinematic hardening parameter
    #    StaticParam.beta      ... Integration parameter (0=expl, 1=impl)
    #    StaticParam. RR       ... Bounding surface radius
    #    CurStress             ... Stress at tn
    #    CurState.eP           ... Plastic strain at tn
    #    CurState.alphaISO     ... Isotropic internal variable at tn
    ##   CurState.alphaKIN     ... Kinematic internal variablen at tn
    #    CurState.Stress0      ... Stress point at unloading
    #
    # Output:
    #    NextStress            ... Stress at tn+1
    #    CurState.eP           ... Plastic strain at tn
    #    CurState.alphaISO     ... Isotropic internal variable at tn+1
    #    CurState.alphaKIN     ... Kinematic internal variablen at tn+1
    #    Cep                   ... Consistent tangent modulus

    #Static Parameters
    G = StaticParam.G
    K = StaticParam.K
    mm = StaticParam.mm
    hh = StaticParam.hh
    beta = StaticParam.beta
    R = StaticParam.RR

    meye = np.eye(3,1); meye[1]=1; meye[2]=1 
    small = 1.0e-10 # Tolerance for small dStrain steps, AKA no stress change
    debugFlag = 1 #Set 0 for true, 1 for false

    #=================================
    # -----INDEX MAPPING AND ROTATIONS

    # Change these to passed values
    #kappaM = kappaM
    res_x, res_y, cen_x, cen_y = resVals(kappaMatrix.shape)

    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    Ce = GetCe(StaticParam)
    Ce = Ce[0:3,0:3].reshape(3,3)
    NextCep = Ce
    
    dev_dStrain = dev(dStrain)
    con_dev_dStrain = (1/2)*dev_dStrain
    vol_dStrain = trace(dStrain)
    con_vol_dStrain = trace((1/2)*dStrain - con_dev_dStrain)
    
    NextStress0 = CurState.Stress0 
    dev_NextStress0 = dev(NextStress0)
    dev_CurStress = dev(CurStress)
    
    CurKappa = CurState.Kappa
    CurPsi = CurState.Psi
    psi = CurState.Psi
    NextKappa = CurKappa # Temporary
    NextH = CurState.H
    NextPsi = CurPsi # Temporary
    NextalphaISO = 0
    Iter_H = 0
    Iter_Psi = 0
    

    #=================================
    # -----UNLOADING CHECK
    numerator = innerProduct((-(1 + CurKappa)*dev_CurStress - CurKappa*(1 + CurKappa)*(dev_CurStress - dev_NextStress0)), dev_dStrain, 3)
    denominator = innerProduct(( (1 + CurKappa)*dev_CurStress - CurKappa*dev_NextStress0), (dev_CurStress - dev_NextStress0), 1)

    if np.absolute(denominator) < small:
        loadingCond = 0
    else:
        loadingCond = numerator/denominator

    if loadingCond > 0.0:
        if debugFlag == 0:
            print('Unloading Happened')
        NextStress0 = CurStress
        dev_NextStress0 = dev(NextStress0)
        CurKappa = 10e6
        NextPsi = 2*G
        Unloading = True
        loadingCond = 0
    else:
        loadingCond = 0
        Unloading = False

    #Unloading 
    #=================================

    if loadingCond == 0:
        
        dev_dStrainNorm = normE(dev_dStrain)
        if np.absolute(dev_dStrainNorm) < small:
            NextStress = CurStress

        else:

            #=========================
            # -----Map to Float-Index Deviatoric Space    
            indexNextStress0 = stressToIndex(dev_NextStress0, res_x, res_y, R, sliced=False) # A\Pi float index
            indexCurStress = stressToIndex(dev_CurStress, res_x, res_y, R, sliced=False) # A\Pi float index
            indexDStrain = stressToIndex(dev_dStrain, res_x, res_y, R/(2*G), sliced=False)

            # Map to O\Pi
            indexNextStress0 = APiToOPi(indexNextStress0,res_x) # O\Pi Index float space
            indexCurStress = APiToOPi(indexCurStress,res_x) # O\Pi Index float space
            indexDStrain = APiToOPi(indexDStrain,res_x)
            
            # Find north-bound angle
            north = np.array([0,1]).reshape(2,1)
            if np.all(indexNextStress0 == 0):
                northUnloadingAngle = 0. # If zero vector stress, set angles to 0
            else:
                northUnloadingAngle = angleIndex(north, indexNextStress0) # Angle of unloading stress from north

            # Rotate north-bound
            indexNorthNextStress0 = indexRotator(indexNextStress0, -northUnloadingAngle) # North O\Pi
            #dist = np.sqrt(indexNextStress0[0]**2 + indexNextStress0[1]**2)
            #indexNorthNextStress0 = np.array([0,dist]).reshape(2,1) # A\Pi
            indexNorthCurStress = indexRotator(indexCurStress, -northUnloadingAngle) # North O\Pi Index float space         
            indexNorthDStrain = indexRotator(indexDStrain, -northUnloadingAngle)
            northDStrain = quatHydroRotator(dev_dStrain, northUnloadingAngle)  # dev_dStrain for South aligned CurStress, North aligned Stress0
            
            # Map to A\Pi
            indexNorthNextStress0 = OPiToAPi(indexNorthNextStress0, res_x) # North A\Pi
            indexNorthCurStress = OPiToAPi(indexNorthCurStress, res_x) # North A\Pi
            
            # Find solution layer
            layerK = cen_y - int(np.round(indexNorthNextStress0[1])) # Layer cen_y is at BS, layer 0 at origin
            
            # Hacky way of preventing excessive elasticity, should be improved
            if layerK > cen_y:
                layerK = cen_y
            # Prevent contact with BS for now, still very coarse in that area
            if layerK == cen_y:
                layerK = layerK - 1
            
            
            iK = int(np.round(indexNorthCurStress[0])) # Current x-coorindate
            jK = int(np.round(indexNorthCurStress[1])) # Current y-coordinate
            CurCoord = np.array([iK, jK]).reshape(2,1) # Assemble cuurent coordinate
            CurCoord = CurCoord.clip(0,res_x-1) # Ensure current coordinate is bounded within BS
            
            # Tune search method for performance
            # Only search outside of current cell if it is neccesary
            max_dStress = 2*G*dev_dStrainNorm # Maximum stress that could be produced by the strain increment
            travelFactor = 1 # Account for angled cell crossing/partials, could be analytical
            resolutionFactor = res_x/R # Account for changing solution matrix cell size
            tol_cell = 1 # If the strain increments max_cells can cross more than tol_cell, use full search
            max_cells = travelFactor*resolutionFactor*max_dStress # Maximum crossable cells due to strain
                
            if max_cells > tol_cell:
                # ========================        
                # -----Search for H' & \psi pair
                # Find paired \psi and H' values to satisfy formula 
                # Search line for \psi and H' pairs
                #rr, cc = psiSearch(northDStrain, indexNorthCurStress, res_x, res_y, R)   
                rr, cc = psiSearch_lite(indexNorthDStrain,indexNorthCurStress,res_x)
                rr = rr.clip(0,res_x-1)
                cc = cc.clip(0,res_x-1)
                
                cartK = np.array(kappaMatrix[rr, cc, layerK])
                CurKappa = float(cartK[0])
                CurH = hh * (CurKappa)**mm
                
                search = 'sequential'
                if search == 'sequential':
                
                    tol_R = 1e-3 # Tolerance for equation 37, major speed increase when looser
                    hold_R = 10e10 # Initialize tolerance comparison
                    Iter_Psi = 0 # Reset psi-search iterations

                    stop = min(int(np.ceil(max_cells)), res_x) # How many points to evaluate on search line (more needed for larger steps/finer resolution)
                    stop = np.linspace(0, stop-1, stop)
                    hold = False
                    hold_psi_one = -1
                    for s in stop:
                        Iter_Psi = s+1 # Iteration of search function

                        # Exit if we are evaluating a value that isn't within our coordinate list
                        if int(s) > len(cartK)-1:
                            break

                        if len(cartK) == 1:
                            # When you are at the BS, only one cell in search-line
                            NextKappa = float(cartK)   
                        else:
                            NextKappa = float(cartK[int(s)]) # H' value of NextStress cell

                        # Compute values for coordinate
                        NextH = float(hh*(NextKappa)**(mm)) # Compute hardening moduli

                        if NextH == 0:
                            NextH = small

                        psi = (2*G) / (1 + 3*G*((1-beta)/NextH + beta/NextH)) # \psi value of specific \psi-H' pair

                        # Solve base equation, zero res_R means pair matches true function (i.e. no error)
                        res_R = np.abs(vectorNorm(dev_CurStress + psi*con_dev_dStrain + NextKappa*(dev_CurStress + psi*con_dev_dStrain - dev_NextStress0),1) - R)

                        # If in an acceptable range of true solution, break loop to save time
                        if res_R <= tol_R:
                            Iter_Psi = -1*int(s) # Shows break graphically
                            hold = False
                            break

                        #Hold best solution if one below tolerance isn't found
                        if res_R < hold_R:
                            hold_NextKappa = NextKappa
                            hold_NextH = NextH
                            #if hold_psi_one == -1:
                            #    hold_psi_one = psi
                            #hold_psi_two = hold_psi_one
                            hold_psi_one = psi
                            hold_Iter_Psi = Iter_Psi
                            hold_R = res_R
                            hold = True

                    # If tolerances were not met, give the best pair found
                    if hold == True:
                        NextKappa = hold_NextKappa 
                        NextH = hold_NextH
                        #psi = (hold_psi_one + hold_psi_two)/2
                        psi = hold_psi_one
                        Iter_Psi = hold_Iter_Psi
                        res_R = hold_R
                        
                elif search == 'bounded':
                    
                    def min_target(x):
                        # Minimizing the residual of this function reveals best solution
                        # x: Point in search list to evaluate, scalar

                        # Discretize input for use in search list
                        idx = int(np.round(x))
                        if idx >= len(rr):
                            idx = len(rr)-1 # Clamp

                        # Choose coordinate to evaluate
                        j = rr[idx]
                        i = cc[idx]

                        # Cooridinate specific values
                        NextKappa = np.array(kappaM[j,i,layerK])
                        NextH = float(hh*(NextKappa)**(mm)) # Compute hardening moduli
                        if NextH == 0:
                            NextH = small # Prevent div\0 issues

                        # Coordinate pair specific \psi value
                        psi = (2*G) / (1 + 3*G*((1-beta)/NextH + beta/NextH))

                        # Solve base equation, zero res_R means pair matches true function (i.e. no error)
                        res_R = np.abs(vectorNorm(dev_CurStress + psi*dev_dStrain + NextKappa*(dev_CurStress + psi*dev_dStrain - dev_NextStress0),1) - R)

                        return res_R
            
                    # Use bounded Golden/Brent-Search (1D, fairly unbiased regarding plasticity/elasticity)
                    #from scipy.optimize import minimize_scalar
                    ext = (0, min(max_cells,len(rr))) # Range to search
                    opt = {'maxiter': 10} # Maximum iterations to allow
                    sol = scipy.optimize.minimize_scalar(min_target, bounds=ext, options = opt, method='bounded') # Minimize residual function
                    
                    # Back-compute values from best residual location
                    idx = int(np.round(sol.x))
                    if idx >= len(rr):
                        idx = idx-1
                    NextKappa = np.array(kappaM[rr[idx],cc[idx],k])
                    NextH = float(hh*NextKappa**mm)
                    psi = (2*G) / (1 + 3*G*((1-beta)/NextH + beta/NextH))
                    res_R = np.abs(vectorNorm(dev_CurStress + psi*dev_dStrain + NextKappa*(dev_CurStress + psi*dev_dStrain - dev_NextStress0),1) - R)
                    Iter_Psi = sol.nfev
                    # fun = sol.fun # Objective function
            else:
                # When we do not expect to leave current cell we take the current coordinates
                # \kappa value to be NextKappa. Very fast, favors elasticity
                #CurKappa = np.array(kappaMatrix[CurCoord[1], CurCoord[0], layerK]) # Determine current H', could pass between time-step
                CurKappa = np.array(kappaMatrix[jK, iK, layerK])
                CurH = hh * (CurKappa)**mm
                
                NextKappa = CurKappa # H' value of NextStress cell  
                NextH = (NextKappa*hh)**(mm) # Compute hardening moduli
                psi = (2*G) / (1 + 3*G*((1-beta)/NextH + beta/NextH)) # \psi value of specific \psi-H' pair

                # Solve base equation, zero res_R means pair matches true function (i.e. no error)
                res_R = np.abs(vectorNorm(dev_CurStress + psi*con_dev_dStrain + NextKappa*(dev_CurStress + psi*con_dev_dStrain - dev_NextStress0),1) - R)
                Iter_Psi = 1 # Set psi-search iterations
                
            Iter_H = res_R
            Iter_H = layerK

                
        #==================================
        # -----Solve Constitutive Equation
        NextStress = CurStress + K*vol_dStrain*meye + psi*con_dev_dStrain

        # Update State
        state = namedtuple('state', ['eP','alphaISO', 'Stress0', 'Iter_H', 'Iter_Psi', 'Kappa', 'H', 'Psi'])
        NextState = state(0, NextalphaISO, NextStress0, Iter_H, Iter_Psi, NextKappa, NextH, psi)

        Ivol = GetIvol()
        Idev = GetIdev()
        Ivol = Ivol[0:3,0:3].reshape(3,3)
        Idev = Idev[0:3,0:3].reshape(3,3)
        NextCep = 3*K*Ivol + (1/2)*psi*Idev

    return NextStress, NextState, NextCep

In [61]:
def psiBSDriver(LoadCase):
    # Precomputed Matrix Enabled BoundingSurface J2 with kinematic hardening 
    # Base driver by Pedro Arduino, Mar. 22 2019
    # Copyright Arduino Computational Geomechanics Group
    # Precomputed solution matrice version by Justin Bonus, Aug. 2019
    #
    #
    # LoadCase:
    #    1 ... proportionally increasing strain
    #    2 ... cyclic strain
    #    3 ... proportionally increasing stress
    #    4 ... cyclic stress
    #
    # ======  LOADING CASES ==================================================
    
    import numpy as np
    from collections import namedtuple
    from skimage.draw import line
    
    nPoints = n
    loadFactor = f
    ## Switch for LoadCases:
    ## Pseudo-switch created by using python dictionary to hold LoadCase functions
    def case_one():
        # Monotonic strain
        case_one.time   = np.linspace(0,1,nPoints+1)
        case_one.strain = np.array([ 0.05, -0.015, -0.015]).reshape(3,1) * loadFactor * case_one.time
        case_one.StressDriven = 0
        return case_one
    
    def case_two():
        # Cyclic strain
        nCycles = 3
        omega = 0.15
        case_two.time = np.linspace(0,nCycles*2*np.pi/omega,nCycles*nPoints+1)
        case_two.strain = np.array([0.050, 0.0, -0.050]).reshape(3,1) * f * np.sin( omega*case_two.time )  #0.045
        case_two.StressDriven = 0
        return case_two
    
    def case_three():
        # Monotonic stress
        case_three.time   = np.linspace(0,1,nPoints+1)
        case_three.stress = np.array([ 0.1, 0.0, 0.0]).reshape(3,1) * f * case_three.time
        case_three.StressDriven = 1
        return case_three
    
    def case_four():
        # Cyclic stress
        nCycles = 3
        omega   = 0.15
        case_four.time   = np.linspace(0, nCycles*2*np.pi/omega, nCycles*nPoints+1)
        case_four.stress = np.array([[0.045],
                                     [0.00],
                                     [-0.045]])*f*np.sin( omega*case_four.time ) + 0.0*np.array([1,0,-1]).reshape(3,1)*np.ones( case_four.time.shape )            
        case_four.StressDriven = 1         
        return case_four
    
    def case_five():
        # Spiral strain
        Su = 0.061 #Undrained shear strength, MPa
        R = Su*(8/3)**0.5
        nRevs = 10/3
        nCycles = 3
        omega   = 0.15
        case_five.time = np.linspace(0, nCycles*2*np.pi/omega, nCycles*nPoints)

        # Spiral
        p = np.linspace(0,nRevs*2*np.pi,nCycles*nPoints)
        a = 3*R
        k = R*np.sqrt(2)/2
        x=a*np.e**(k*p)*np.sin(p)
        y=a*np.e**(k*p)*np.cos(p)
        y = y-a; y = -y # Center and flip

        case_five.strain = np.array([[np.sqrt(2/3)*0.01],[0],[0]])*y + np.array([[0],[(1/2)*0.01],[-(1/2)*0.01]])*x
        case_five.strain = np.append(case_five.strain, np.flip(case_five.strain,1),1).reshape(3,2*nCycles*nPoints)
        case_five.time = np.append(case_five.time,case_five.time[::-1])
        case_five.StressDriven = 0
        return case_five
    
    def case_six():
        # Spiral Stress
        Su = 0.061 #Undrained shear strength, MPa
        R = Su*(8/3)**0.5
        nRevs = 10/3
        nCycles = 3
        omega   = 0.15
        case_six.time = np.linspace(0, nCycles*2*np.pi/omega, nCycles*nPoints)

        # Spiral
        p = np.linspace(0,nRevs*2*np.pi,nCycles*nPoints)
        a = 3*R
        k = R*np.sqrt(2)/2
        x=a*np.e**(k*p)*np.sin(p)
        y=a*np.e**(k*p)*np.cos(p)
        y = y-a; y = -y # Center and flip

        case_six.stress = np.array([[np.sqrt(2/3)*0.05],[0],[0]])*y + np.array([[0],[(1/2)*0.05],[-(1/2)*0.05]])*x
        case_six.stress = np.append(case_six.stress, np.flip(case_six.stress,1),1).reshape(3,2*nCycles*nPoints)
        case_six.time = np.append(case_six.time,case_six.time[::-1])
        case_six.StressDriven = 1
        return case_six
    
    case_switcher = {
        1: case_one,
        2: case_two,
        3: case_three,
        4: case_four,
        5: case_five,
        6: case_six
    }    

    case = case_switcher.get(LoadCase, lambda: "Invalid LoadCase")
    case() #Runs the LoadCase function. Creates: case.time, case.strain | case.stress, case.StressDriven
    time, StressDriven = case.time, case.StressDriven 
    if StressDriven == 1:
        stress = case.stress
        strain = np.zeros((3,1)) #initialize empty 3x1 strain numpy array for stress-driven scenario
    else:
        strain = case.strain
        stress = np.zeros((3,1)) #initialize empty 3x1 stress numpy array for strain-driven scenario
    Stress0 = np.zeros((3,1)) #Initialize first 'unloading' point

    # ========================================================================
    # ---- MATERIAL PARAMETERS
    # Static Parameters
    E = 20 #Elastic Modulus  MPa
    v= 0.49 #Poissons ratio, less than 0.5 to allow compresibility
    G = E/(2*(1+v)) #Shear modulus
    K = E/(3*(1-2*v)) #Bulk modulus
    Kmod = 0 #Isotropic Hardening
    Su = 0.061 #Yield stress in 1-D tension test MPa
    hh = G #kinematic hardening parameter
    mm = 1.0 #kinematic hardening parameter
    beta = 0.5 #midpoint integration
    RR = np.sqrt(8/3)*Su # Make RR

    # Class structure
    static = namedtuple('StaticParam',['E','v','G','K','Kmod','Su','hh','mm','beta','RR'])
    StaticParam = static(E,v,G,K,Kmod,Su,hh,mm,beta,RR)

    # ========================================================================
    # ---- LOAD PRECOMPUTED MATRICES
    # x = 201
    # y = x
    # kappaM = loadKappa(x, y)
    res_x, res_y, cen_x, cen_y = resVals(kappaM.shape) 
    
    # ========================================================================
    # ---- INITIAL CONDITIONS

    # Initialize the state variables
    if StressDriven == True:
        IniStress = 0.0*np.array([1, 1, 1]).reshape(3,1)
        Ce = GetCe(StaticParam)
        Ce = Ce[0:3,0:3].reshape(3,3)
        IniStrain = np.linalg.solve(Ce, IniStress) #Check if GetCe compacts to nxn
    else:
        IniStress =  0.0*np.array([1, 1, 1]).reshape(3,1)
        Ce = GetCe(StaticParam)
        Ce = Ce[0:3,0:3].reshape(3,3)
        IniStrain =  0.0*np.array([1, 1, 1]).reshape(3,1)   

    eP = 0.0*(np.array([1, 1, 1]).reshape(3,1))
    alphaISO = 0.0  
    Stress0 = 0.0*(np.array([1, 1, 1]).reshape(3,1))
    Kappa = 1e6 #maxKappa
    H = StaticParam.hh*Kappa**StaticParam.mm
    Psi = 2*StaticParam.G
    
    # Structure for IniState (initial state parameters, static) and CurState (changing state parameters)
    state = namedtuple('state', ['eP','alphaISO','Stress0','Kappa','H','Psi'])
    IniState = state(eP, alphaISO, Stress0, Kappa, H, Psi)

    # For first iteration
    CurStress = IniStress
    CurStrain = IniStrain
    CurState  = IniState

    # Variables used for plotting
    kappa, H, psi = [],[],[]
    C = []
    strain[:,0] = CurStrain.T - IniStrain.T 
    stress[:,0] = CurStress.T - IniStress.T #Include -ini ?
    Stress0[:,0] = CurStress.T
    alphaISO_plot, j2_plot, j2e_plot, stress_var_plot, stress_var2_plot = [],[],[],[],[] #Initiliaze list format
    alphaISO_plot.append(0) #Python list appending for easy data addition, could be faster if array size was initialized entirely
    j2_plot.append(0)
    j2e_plot.append(0)
    stress_var_plot.append(0)
    Iter_H = np.zeros(time.shape)
    Iter_Psi = np.zeros(time.shape)
    Iter = np.zeros(time.shape)
    maxKappa = 10e5 # Maximum considered \kappa value, stand in for infinity
    maxH = StaticParam.hh*maxKappa**StaticParam.mm # Hardening function dependent
    maxPsi = 2*StaticParam.G
    kappa.append(maxKappa) # Assumed to start at unloading point 
    H.append(maxH) # Assumed to start at unloading point
    psi.append(maxPsi) # Assumed to start at unloading point
    C.append(Ce)

    # ========================================================================
    # ---- COMPUTATION CYCLES

    if StressDriven == False:
        # Strain-driven
        for i in range(1, (len(strain[0]) )):
            
            # Input strain
            NextStrain = strain[:,i] + IniStrain.T
            dStrain = (strain[:,i] - strain[:, i-1]).reshape(3,1) #Driving variable
            NextStress, NextState, NextCep = psiBSRadialMap(dStrain, StaticParam, CurStress, CurState, kappaM)

            # Update Stress, Strain, and State
            CurStress = NextStress
            CurState = NextState
            
            # Variables created for plotting purposes
            stress = np.append(stress, CurStress, 1)
            pseudoStress = stress
            Stress0 = np.append(Stress0, CurState.Stress0, 1)
            alphaISO_plot.append(CurState.alphaISO)
            j2_plot.append(0)
            stress_var_plot.append(np.sqrt(2*j2_plot[i])*np.sqrt(3/2)*np.sign(stress[0,i] - stress[1,i]))
            stress_var2_plot.append((stress[0,i] - stress[1,i]))
            Iter_H[i] = CurState.Iter_H
            Iter_Psi[i] = CurState.Iter_Psi
            kappa.append(CurState.Kappa)
            H.append(CurState.H)
            psi.append(CurState.Psi)
            C.append(NextCep)
            
    elif StressDriven == True:          
        # StressDriven driver
        # set tolerance value for iterative procedure(s)
        TOLERANCE = 1e-10 
        pseudoStress = IniStress
        for i in range(0, len(stress[0])-1):

            # initialize strain epsilon_{n+1}^{(0)} = eps_{n} using the old state
            # (this is the initial approximation for eps_{n+1}
            if i == 0:
                # special settings for initial values at t_1
                NextStrain = np.array([0,0,0]).reshape(3,1)
                dStrain = np.array([0,0,0]).reshape(3,1)
                CurState = IniState
            else:
                NextStrain = CurStrain
                dStrain = np.array([0,0,0]).reshape(3,1)

            NextStress, NextState, Cep = psiBSRadialMap(dStrain, StaticParam, CurStress, CurState, kappaM)

            RR = stress[:, i].reshape(3,1) - NextStress
            RR = RR.reshape(3,1)
            RR0 = normS(RR)

            # reset iteration counter
            kk = 0
            # iterate until convergence
 
            while normS(RR)/RR0 > TOLERANCE:
                
                # update strain from eps_{n+1}^{(k)} to eps_{n+1}^{(k+1)}
                dStrain = np.linalg.solve(Cep, RR)
                NextStrain = NextStrain + dStrain

                # compute material response for estimated strain state
                # NOTE: the state variables are taken at t_n
                NextStress, NextState, Cep = psiBSRadialMap(dStrain, StaticParam, CurStress, CurState, kappaM)

                # check for equilibrium
                RR = stress[:,i].reshape(3,1) - NextStress
                RR = RR.reshape(3,1)
                kk = kk + 1
                # emergence exit if procedure does not converge            
                if kk > 3:
                    kk = kk
                    #print('procedure slow to converge. Error : ', normS( RR )/RR0)
                
                if kk > 20:
                    #print('procedure did not converge. Error : ', normS( RR )/RR0)
                    #print('YOUR TANGENT Cep IS WRONG', normS( RR )/RR0)
                    break

                Iter[i] = kk
                CurStress = NextStress
                CurState = NextState
                


            # Update State variables for next step
            CurStress = NextStress
            CurStrain = NextStrain
            CurState  = NextState
            
            # Variables created for plotting purposes
            strain = np.append(strain, CurStrain, 1)
            pseudoStress = np.append(pseudoStress, CurStress, axis=1)
            Stress0 = np.append(Stress0, CurState.Stress0, 1)
            alphaISO_plot.append(CurState.alphaISO)
            j2_plot.append(0)
            stress_var_plot.append(np.sqrt(2*j2_plot[i])*np.sqrt(3/2)*np.sign(stress[0,i] - stress[1,i]))
            stress_var2_plot.append((stress[0,i] - stress[1,i]))
            Iter_H[i] = CurState.Iter_H
            Iter_Psi[i] = CurState.Iter_Psi
            kappa.append(CurState.Kappa)
            H.append(CurState.H)
            psi.append(CurState.Psi)
            C.append(Cep)
            
    DriverOutput = namedtuple('DriverOutput',['StaticParam','time','strain','stress','pseudoStress','alphaISO','j2','stress_var','stress_var2', 'Stress0', 'Iter', 'Iter_H', 'Iter_Psi', 'kappa', 'H', 'psi', 'C'])
    DriverOutput = DriverOutput(StaticParam, time, strain, stress, pseudoStress, alphaISO_plot, j2_plot, stress_var_plot, stress_var2_plot, Stress0, Iter, Iter_H, Iter_Psi, kappa, H, psi, C)

    return DriverOutput
    # =========================================================================

# Precomputed

3x1 driven, only features pure-strain and pure-stress driving

In [7]:
def kappaBSRadialMap(dStr, StaticParam, CurStrain, CurStress, CurState, StressDriven):
    # Radial Return Algorithm for J2 Bounding Surface Plasticity Model
    # Original model by Borjas & Amies 1994
    # Base radialmap by Pedro Arduino, Mar. 22 2019
    # Copyright Arduino Computational Geomechanics Group
    # Precomputed matrix enabled version by Justin Bonus, Jul. 2019
    #
    #
    # Input:
    #    dStrain               ... Strain differential, tn to tn+1
    #    StaticParam.E         ... Young's modulus
    #    StaticParam.v         ... poissons ratio
    #    StaticParam.G         ... Shear modulus
    #    StaticParam.K         ... Bulk modulus
    #    StaticParam.hh        ... Kinematic Hardening parameter
    #    StaticParam.mm        ... Kinematic hardening parameter
    #    StaticParam.beta      ... Integration parameter (0=expl, 1=impl)
    #    StaticParam. RR       ... Bounding surface radius
    #    CurStress             ... Stress at tn
    #    CurState.eP           ... Plastic strain at tn
    #    CurState.alphaISO     ... Isotropic internal variable at tn
    ##   CurState.alphaKIN     ... Kinematic internal variablen at tn
    #    CurState.Stress0      ... Stress point at unloading
    #
    # Output:
    #    NextStress            ... Stress at tn+1
    #    CurState.eP           ... Plastic strain at tn
    #    CurState.alphaISO     ... Isotropic internal variable at tn+1
    #    CurState.alphaKIN     ... Kinematic internal variablen at tn+1
    #    Cep                   ... Consistent tangent modulus

    #Static Parameters
    G = StaticParam.G
    K = StaticParam.K
    mm = StaticParam.mm
    hh = StaticParam.hh
    beta = StaticParam.beta
    R = StaticParam.RR

    meye = np.eye(3,1); meye[1]=1; meye[2]=1 
    small = 1.0e-10 # Tolerance for small dStrain steps, AKA no stress change
    debugFlag = 1 #Set 0 for true, 1 for false

    #=================================
    # -----INDEX MAPPING AND ROTATIONS

    # Change these to passed values
    #res_x = 101
    #res_y = 101
    cen_x = int(np.floor(res_x/2))
    cen_y = int(np.floor(res_y/2))
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    if StressDriven == 0:
        # Strain-driven loading
        #print('Strain-driven')
        dStrain = dStr

        dev_dStrain = dev(dStrain)
        vol_dStrain = trace(dStrain)

        NextStress0 = CurState.Stress0 
        dev_NextStress0 = dev(NextStress0)
        dev_CurStress = dev(CurStress)

        CurKappa = CurState.Kappa
        CurPsi = CurState.Psi
        NextKappa = CurKappa # Temporary
        NextPsi = CurPsi # Temporary

        NextalphaISO = 0

        # Unit-vectors
        hydro = np.array([[np.sqrt(1/3)],[np.sqrt(1/3)],[np.sqrt(1/3)]]) # Hydrostatic axis unit-vector
        north = np.array([[np.sqrt(2/3)],[-np.sqrt(1/6)],[-np.sqrt(1/6)]]) #\Pi-plane north unit-vector
        east = np.array([[0],[np.sqrt(1/2)],[-np.sqrt(1/2)]]) #\Pi-plane east unit-vector
        south = -north

        #=================================
        # -----UNLOADING CHECK
        numerator = innerProduct((-(1 + CurKappa)*dev_CurStress - CurKappa*(1 + CurKappa)*(dev_CurStress - dev_NextStress0)), dev_dStrain, 3)
        denominator = innerProduct(( (1 + CurKappa)*dev_CurStress - CurKappa*dev_NextStress0), (dev_CurStress - dev_NextStress0), 1)

        if np.absolute(denominator) < small:
            loadingCond = 0
        else:
            loadingCond = numerator/denominator

        if loadingCond > 0.0:
            if debugFlag == 0:
                print('Unloading Happened')
            NextStress0 = CurStress
            dev_NextStress0 = dev(NextStress0)
            loadingCond = 0
        else:
            loadingCond = 0

        unloading = True # Set to True in order to enter while loop
        #=================================
        
        if loadingCond == 0:
            if np.all(dev_NextStress0 == 0):
                northUnloadingAngle = 0. # If zero vector stress, set angles to 0
                eastUnloadingAngle = 0.
            else:
                northUnloadingAngle = angleNormal(north, dev_NextStress0) # Angle of unloading stress from north
                eastUnloadingAngle = angleNormal(east, dev_NextStress0) # Angle of unloading stress from east
            if eastUnloadingAngle > np.pi/2:
                northUnloadingAngle = -northUnloadingAngle # Check if west of centerline, adjust

            #=========================
            # -----Rotate North & Map to Index Space      
            indexNextStress0 = stressToIndex(NextStress0, res_x, res_y, R, sliced=False) - cen_y # O\Pi Index float space
            indexCurStress = stressToIndex(CurStress, res_x, res_y, R, sliced=False) - cen_y # O\Pi Index float space
            indexNorthNextStress0 = indexRotator(indexNextStress0, -northUnloadingAngle) + cen_y # A\Pi Index float space
            indexNorthCurStress = indexRotator(indexCurStress, -northUnloadingAngle) + cen_y # A\Pi Index float space         
            northDStrain = quatHydroRotator(dev_dStrain, -northUnloadingAngle)  # dev_dStrain for South aligned CurStress, North aligned Stress0
            #southDStrain = -southNorthDStrain # dev_dStrain for North aligned CurStress, North aligned Stress0

            #=========================
            # -----Access \kappa Matrix
            # Find layer containing appropiate 1D North NextStress0 position
            if indexNorthNextStress0[1] == 0:
                indexNorthNextStress0[1] = 1
            layerK = cen_y - int(np.round(indexNorthNextStress0[1]))
            jK = int(np.round(indexNorthCurStress[1]))
            iK = int(np.round(indexNorthCurStress[0]))
            cartK = kappaM[:, :, layerK] # Holds relevant H' matrix layer
            CurKappa = cartK[jK, iK] # Determine current H', could pass between time-step
            CurH = (hh * CurKappa)**mm

            # ========================        
            # -----Search for H' & \psi pair
            # Find paired psi and NextH values to satisfy formula 

            # Search line for \psi and H' pairs
            rr, cc = psiSearch(northDStrain, indexNorthCurStress, res_x, res_y, R)    
            
            psi = 2*G # Initialize psi
            tol_R = 1e-2 # Tolerance for equation 37, major speed increase when looser
            hold_R = 1e6 # Initialize tolerance comparison eq. 37
            Iter_Psi = 0 # Reset psi-search iterations
            Iter_H = 0 # Redundant, currently used for tracking random  values of interest
 
            # 'stop' is a very important parameter
            # High values will slow the program/increase chance of error
            # Low values will limit the stress movement/increase error
            # Will eventually become 'predictively' set
            stop = 30 # How many points to evaluate on search line (more needed for larger steps/finer resolution)
            stop = np.linspace(0, stop-1, stop) 
            for j, i, s in zip(rr, cc, stop):
                Iter_Psi = s # Iteration of search function
                NextKappa = float(cartK[j,i]) # H' value of NextStress cell
                if NextKappa == 0:
                    # Skip if cell is on/outside of bounding surface
                    continue   
                NextH = float((NextKappa*hh)**(mm)) # Backcompute kappa from hardening function
                psi = (2*G) / (1 + 3*G*(((1-beta)/NextH) + (beta/NextH))) # \psi value of specific \psi-H' pair
                
                # Solve base equation, zero res_R means pair matches true function (i.e. no error)
                res_R = np.abs(np.linalg.norm(dev_CurStress + psi*dev_dStrain + NextKappa*(dev_CurStress + psi*dev_dStrain - dev_NextStress0))-R)

                # If in an acceptable range of true solution, break loop to save time
                if res_R <= tol_R:
                    hold = False
                    break

                #Hold best solution if one below tolerance isn't found
                if res_R < hold_R:
                    hold_NextKappa = NextKappa
                    hold_NextH = NextH
                    hold_psi = psi
                    hold_Iter_Psi = Iter_Psi
                    hold_R = res_R
                    hold = True

            # If tolerances were not met, give the best pair found
            if hold == True:
                NextKappa = hold_NextKappa 
                NextH = hold_NextH
                psi = hold_psi
                Iter_Psi = hold_Iter_Psi
                res_R = hold_R

        #==================================
        # -----Solve Constitutive Equation
        #NextStress = CurStress + K*vol_dStrain*meye + psi*dev_dStrain
        NextStress = CurStress + K*vol_dStrain*meye + (1/2)*psi*dev_dStrain
        NextStrain = CurStrain + vol_dStrain + dev_dStrain
        
        # Update State
        Iter_H = res_R # Redundant
        state = namedtuple('state', ['eP','alphaISO', 'Stress0', 'Iter_H', 'Iter_Psi', 'Kappa', 'H', 'Psi'])
        NextState = state(0, NextalphaISO, NextStress0, Iter_H, Iter_Psi, NextKappa, NextH, psi)
    
        return NextStrain, NextStress, NextState
    
    # -----------------------------------------------------------------------------------------------------------
    # STRESS - DRIVEN    
    elif StressDriven == 1:
        
        # Initialize state
        dev_CurStrain = dev(CurStrain)
        dev_CurStress = dev(CurStress)      
        
        dStress = dStr        
        dev_dStress = dev(dStress)
        vol_dStress = trace(dStress)*np.array([[1/3],[1/3],[1/3]])
        dStress = dev_dStress + vol_dStress
        
        NextStress = CurStress + dStress
        dev_NextStress = dev(NextStress)
        
        NextStress0 = CurState.Stress0 
        dev_NextStress0 = dev(NextStress0)
        norm_dev_NextStress0 = normS(dev_NextStress0)        
        
        # Initialize state parameters
        CurKappa = CurState.Kappa
        CurPsi = CurState.Psi
        NextKappa = CurKappa # Temporary
        NextPsi = CurPsi # Temporary
        NextalphaISO = 0
        
        north = np.array([[np.sqrt(2/3)],[-np.sqrt(1/6)],[-np.sqrt(1/6)]]) #\Pi-plane north unit-vector
        east = np.array([[0],[np.sqrt(1/2)],[-np.sqrt(1/2)]]) #\Pi-plane east unit-vector
        
        #=================================
        # -----UNLOADING CHECK
        unloaded = False
        numerator = innerProduct((-(1 + CurKappa)*dev_CurStress - CurKappa*(1 + CurKappa)*(dev_CurStress - dev_NextStress0)), dev_dStress, 1)
        denominator = innerProduct(( (1 + CurKappa)*dev_CurStress - CurKappa*dev_NextStress0), (dev_CurStress - dev_NextStress0), 1)
        small = 1e-10
        if np.absolute(denominator) < small:
            loadingCond = 0
        else:
            loadingCond = numerator/denominator

        if loadingCond > 0.0:
            # UNLOADED
            # Reset NextStress0 to CurStress position
            if debugFlag == 0:
                print('Unloading Happened')
            NextStress0 = CurStress
            dev_NextStress0 = dev(NextStress0)
            norm_dev_NextStress0 = normS(dev_NextStress0) 
            loadingCond = 0
            unloaded = True
        else:
            # LOADING
            # Maintain NextStress0
            loadingCond = 0
            unloaded = False
        
        if np.all(NextStress0 == 0):
            northUnloadingAngle = 0. # If zero vector unloading stress
            eastUnloadingAngle = 0. # If zero vector unloading stress
        else:
            northUnloadingAngle = angleNormal(north, dev_NextStress0)
            eastUnloadingAngle = angleNormal(east, dev_NextStress0)
        if eastUnloadingAngle > np.pi/2:
            northUnloadingAngle = -northUnloadingAngle # If west of centerline
            
        #============================================
        # ----- Map to Index Space and Rotate North
        # Rotate and map stress state to index notation
        indexCurStress = stressToIndex(dev_CurStress, res_x, res_y, R) - cen_y
        indexNextStress = stressToIndex(dev_NextStress, res_x, res_y, R) - cen_y
        indexNextStress0 = stressToIndex(dev_NextStress0, res_x, res_y, R) - cen_y
        indexNorthCurStress = indexRotator(indexCurStress, -northUnloadingAngle) + cen_y
        indexNorthNextStress = indexRotator(indexNextStress, -northUnloadingAngle) + cen_y
        indexNorthNextStress0 = indexRotator(indexNextStress0, -northUnloadingAngle) + cen_y
          
        if indexNorthNextStress0[1] <= 0:
            indexNorthNextStress0[1] = 1

        #===========================================
        # ----- Access H' Matrix
        layer = cen_y - int(indexNorthNextStress0[1])  # Relevant H' matrix layer for NextStress0
        if unloaded == True:
            maxKappa = 1e5
            CurKappa = maxKappa
        elif unloaded == False:
            CurKappa = float(kappaM[int(indexNorthCurStress[1]),int(indexNorthCurStress[0]), layer])
        CurH = float((CurKappa*hh)**(mm))
        NextKappa = float(kappaM[int(indexNorthNextStress[1]),int(indexNorthNextStress[0]), layer])
        NextH = float((NextKappa * hh)**(mm))
        trap = (1 + 3*G*(((1-beta)/CurH) + (beta/float(NextH))))
        psi = (2*G) / trap
        
        #============================================
        # ----- Solve Constitutive Equation
        dev_dStrain = (dev_dStress * trap) / (2*G)
        #dev_dStrain = dev_dStress/psi
        #vol_dStrain = (vol_dStress) / (3*K)
        vol_dStrain = (dStress - psi*dev_dStrain) / (K)
        
        # Issue in convert2StressLike() probably, should fix 
        NextStrain = CurStrain + vol_dStrain + 2*dev_dStrain # Cylic
        #NextStrain = CurStrain + vol_dStrain + dev_dStrain # Monotonic

        Iter_H = 0 # No iterations required in stress-driving
        Iter_Psi = 0 # No iterations required in stress-driving
        
        # Update State
        state = namedtuple('state', ['eP','alphaISO', 'Stress0', 'Iter_H', 'Iter_Psi', 'Kappa', 'H', 'Psi'])
        NextState = state(0, NextalphaISO, NextStress0, Iter_H, Iter_Psi, NextKappa, NextH, psi)
    
        return NextStrain, NextStress, NextState

In [12]:
def precomputedBSDriver(LoadCase):
    # Precomputed Matrix Enabled BoundingSurface J2 with kinematic hardening 
    # Base driver by Pedro Arduino, Mar. 22 2019
    # Copyright Arduino Computational Geomechanics Group
    # Precomputed solution matrice version by Justin Bonus, Aug. 2019
    #
    #
    # LoadCase:
    #    1 ... proportionally increasing strain
    #    2 ... cyclic strain
    #    3 ... proportionally increasing stress
    #    4 ... cyclic stress
    #
    # ======  LOADING CASES ==================================================
    
    import numpy as np
    from collections import namedtuple
    from skimage.draw import line
    
    nPoints = 100

    ## Switch for LoadCases:
    ## Pseudo-switch created by using python dictionary to hold LoadCase functions
    def case_one():
        # Monotonic strain
        case_one.time   = np.linspace(0,1,nPoints+1)
        case_one.strain = np.array([ 0.05, -0.015, -0.015]).reshape(3,1) * case_one.time
        case_one.StressDriven = 0
        return case_one
    
    def case_two():
        # Cyclic strain
        nCycles = 3
        omega = 0.15
        case_two.time = np.linspace(0,nCycles*2*np.pi/omega,nCycles*nPoints+1)
        case_two.strain = np.array([0.050, 0.0, -0.050]).reshape(3,1) * np.sin( omega*case_two.time )  #0.045
        case_two.StressDriven = 0
        return case_two
    
    def case_three():
        # Monotonic stress
        case_three.time   = np.linspace(0,1,nPoints+1)
        case_three.stress = np.array([ 0.1, 0.0, 0.0]).reshape(3,1) * case_three.time
        case_three.StressDriven = 1
        return case_three
    
    def case_four():
        # Cyclic stress
        nCycles = 3
        omega   = 0.15
        case_four.time   = np.linspace(0, nCycles*2*np.pi/omega, nCycles*nPoints+1)
        case_four.stress = np.array([[0.05],
                                     [0.00],
                                     [-0.05]])*np.sin( omega*case_four.time ) + 0.0*np.array([1,1,1]).reshape(3,1)*np.ones( case_four.time.shape )            
        case_four.StressDriven = 1         
        return case_four
    
    def case_five():
        # Spiral strain
        Su = 0.061 #Undrained shear strength, MPa
        R = Su*(8/3)**0.5
        nRevs = 10/3
        nCycles = 3
        omega   = 0.15
        case_five.time = np.linspace(0, nCycles*2*np.pi/omega, nCycles*nPoints)

        # Spiral
        p = np.linspace(0,nRevs*2*np.pi,nCycles*nPoints)
        a = 3*R
        k = R*np.sqrt(2)/2
        x=a*np.e**(k*p)*np.sin(p)
        y=a*np.e**(k*p)*np.cos(p)
        y = y-a; y = -y # Center and flip

        case_five.strain = np.array([[np.sqrt(2/3)*0.01],[0],[0]])*y + np.array([[0],[(1/2)*0.01],[-(1/2)*0.01]])*x
        case_five.strain = np.append(case_five.strain, np.flip(case_five.strain,1),1).reshape(3,2*nCycles*nPoints)
        case_five.time = np.append(case_five.time,case_five.time[::-1])
        case_five.StressDriven = 0
        return case_five
    
    def case_six():
        # Spiral Stress
        Su = 0.061 #Undrained shear strength, MPa
        R = Su*(8/3)**0.5
        nRevs = 10/3
        nCycles = 3
        omega   = 0.15
        case_six.time = np.linspace(0, nCycles*2*np.pi/omega, nCycles*nPoints)

        # Spiral
        p = np.linspace(0,nRevs*2*np.pi,nCycles*nPoints)
        a = 3*R
        k = R*np.sqrt(2)/2
        x=a*np.e**(k*p)*np.sin(p)
        y=a*np.e**(k*p)*np.cos(p)
        y = y-a; y = -y # Center and flip

        case_six.stress = np.array([[np.sqrt(2/3)*0.05],[0],[0]])*y + np.array([[0],[(1/2)*0.05],[-(1/2)*0.05]])*x
        case_six.stress = np.append(case_six.stress, np.flip(case_six.stress,1),1).reshape(3,2*nCycles*nPoints)
        case_six.time = np.append(case_six.time,case_six.time[::-1])
        case_six.StressDriven = 1
        return case_six
    
    case_switcher = {
        1: case_one,
        2: case_two,
        3: case_three,
        4: case_four,
        5: case_five,
        6: case_six
    }    

    case = case_switcher.get(LoadCase, lambda: "Invalid LoadCase")
    case() #Runs the LoadCase function. Creates: case.time, case.strain | case.stress, case.StressDriven
    time, StressDriven = case.time, case.StressDriven 
    if StressDriven == 1:
        stress = case.stress
        strain = np.zeros((3,1)) #initialize empty 3x1 strain numpy array for stress-driven scenario
    else:
        strain = case.strain
        stress = np.zeros((3,1)) #initialize empty 3x1 stress numpy array for strain-driven scenario
    Stress0 = np.zeros((3,1)) #Initialize first 'unloading' point

    # ========================================================================
    # ---- MATERIAL PARAMETERS
    # Static Parameters
    E = 20 #Elastic Modulus  MPa
    v= 0.49 #Poissons ratio, less than 0.5 to allow compresibility
    G = E/(2*(1+v)) #Shear modulus
    K = E/(3*(1-2*v)) #Bulk modulus
    Kmod = 0 #Isotropic Hardening
    Su = 0.061 #Yield stress in 1-D tension test MPa
    hh = G #kinematic hardening parameter
    mm = 1.0 #kinematic hardening parameter
    beta = 0.5 #midpoint integration
    RR = np.sqrt(8/3)*Su # Make RR

    # namedtuple used to organzie related variables, similar to a structure
    static = namedtuple('StaticParam',['E','v','G','K','Kmod','Su','hh','mm','beta','RR'])
    StaticParam = static(E,v,G,K,Kmod,Su,hh,mm,beta,RR)

    # ========================================================================
    # ---- LOAD PRECOMPUTED MATRICES
    #res_x = 
    #res_y = 
    # Currently matrices are loaded in the notebook before reaching the driver,
    # it's unnecesary to reload them.
    # Read \kappa Matrix
    #fn = 'kappaMatrix_' + str(res_x) + '_' + str(res_y) + '.txt'
    #kappaM = np.loadtxt(fn).reshape((res_y,res_x,int(np.floor(res_y/2))+1)) # Returns 3D array
    #-------------------------------------------------------------------------
    # Read H' Matrix
    #fn = 'hardeningMatrix_' + str(res_x) + '_' + str(res_y) + '_' + str(hh) + '_'+ str(mm) + '.txt'
    #hardeningM = np.loadtxt(fn).reshape((res_y,res_x,int(np.floor(res_y/2))+1)) # Returns 3D array
    #-------------------------------------------------------------------------
    # Read \psi Matrix
    #cen_y = int(np.floor(res_y/2))
    #fn = 'psiMatrix_' + str(res_x) + '_' + str(res_y) + '_' + str(hh) + '_' + str(mm) + '.txt'
    #depth = res_y*(cen_y)
    #depth = res_y*(cen_y+2)
    #psiM = np.loadtxt(fn).reshape((res_y,res_x,depth)) # Returns 3D array
    
    # ========================================================================
    # ---- INITIAL CONDITIONS

    # Initialize the state variables
    if StressDriven == True:
        IniStress = 0.0*np.array([1, 1, 1]).reshape(3,1)
        IniStrain =  0.0*np.array([1, 1, 1]).reshape(3,1)
    else:
        IniStress =  0.0*np.array([1, 1, 1]).reshape(3,1)
        IniStrain =  0.0*np.array([1, 1, 1]).reshape(3,1)   

    eP = 0.0*(np.array([1, 1, 1]).reshape(3,1))
    alphaISO = 0.0  
    Stress0 = 0.0*(np.array([1, 1, 1]).reshape(3,1))
    Kappa = 1e6 #maxKappa
    H = StaticParam.hh*Kappa**StaticParam.mm
    Psi = 2*StaticParam.G
    
    # Structure for IniState (initial state parameters, static) and CurState (changing state parameters)
    state = namedtuple('state', ['eP','alphaISO','Stress0','Kappa','H','Psi'])
    IniState = state(eP, alphaISO, Stress0, Kappa, H, Psi)

    # For first iteration
    CurStress = IniStress
    CurStrain = IniStrain
    CurState  = IniState

    # Variables used for plotting
    kappa, H, psi = [],[],[]
    strain[:,0] = CurStrain.T - IniStrain.T 
    stress[:,0] = CurStress.T - IniStress.T #Include -ini ?
    Stress0[:,0] = CurStress.T
    alphaISO_plot, j2_plot, j2e_plot, stress_var_plot, stress_var2_plot = [],[],[],[],[] #Initiliaze list format
    alphaISO_plot.append(0) #Python list appending for easy data addition, could be faster if array size was initialized entirely
    j2_plot.append(0)
    j2e_plot.append(0)
    stress_var_plot.append(0)
    Iter_H = np.zeros(time.shape)
    Iter_Psi = np.zeros(time.shape)
    
    maxKappa = 10e5 # Maximum considered \kappa value, stand in for infinity
    maxH = StaticParam.hh*maxKappa**StaticParam.mm # Hardening function dependent
    maxPsi = 2*StaticParam.G
    kappa.append(maxKappa) # Assumed to start at unloading point 
    H.append(maxH) # Assumed to start at unloading point
    psi.append(maxPsi) # Assumed to start at unloading point


    # ========================================================================
    # ---- COMPUTATION CYCLES

    if StressDriven == False:
        # Strain-driven
        for i in range(1, (len(strain[0]) )):
            
            # Input strain
            NextStrain = strain[:,i] + IniStrain.T
            dStrain = (strain[:,i] - strain[:, i-1]).reshape(3,1) #Driving variable
            NextStrain, NextStress, NextState = kappaBSRadialMap(dStrain, StaticParam, CurStrain, CurStress, CurState, StressDriven)

            # Update Stress, Strain, and State
            CurStress = NextStress
            CurState = NextState
            
            # Variables created for plotting purposes
            stress = np.append(stress, CurStress, 1)
            Stress0 = np.append(Stress0, CurState.Stress0, 1)
            alphaISO_plot.append(CurState.alphaISO)
            j2_plot.append(0)
            stress_var_plot.append(np.sqrt(2*j2_plot[i])*np.sqrt(3/2)*np.sign(stress[0,i] - stress[1,i]))
            stress_var2_plot.append((stress[0,i] - stress[1,i]))
            Iter_H[i] = CurState.Iter_H
            Iter_Psi[i] = CurState.Iter_Psi
            kappa.append(CurState.Kappa)
            H.append(CurState.H)
            psi.append(CurState.Psi)
            
    elif StressDriven == True:
        # Stress-driven
        for i in range(1, len(stress[0]) ):
            
            # Input stress
            NextStress = stress[:,i] + IniStress.T
            dStress = (stress[:,i] - stress[:, i-1]).reshape(3,1)
            NextStrain, NextStress, NextState = kappaBSRadialMap(dStress, StaticParam, CurStrain, CurStress, CurState, StressDriven)
            
            # Update Stress, Strain, and State            
            CurStrain = NextStrain
            CurStress = NextStress
            CurState = NextState
            
            # Variables created for plotting purposes
            strain = np.append(strain, CurStrain, 1)
            Stress0 = np.append(Stress0, CurState.Stress0, 1)
            alphaISO_plot.append(CurState.alphaISO)
            j2_plot.append(0)
            stress_var_plot.append(np.sqrt(2*j2_plot[i])*np.sqrt(3/2)*np.sign(stress[0,i] - stress[1,i]))
            stress_var2_plot.append((stress[0,i] - stress[1,i]))
            Iter_H[i] = CurState.Iter_H
            Iter_Psi[i] = CurState.Iter_Psi
            kappa.append(CurState.Kappa)
            H.append(CurState.H)
            psi.append(CurState.Psi)
            
            
    DriverOutput = namedtuple('DriverOutput',['StaticParam','time','strain','stress','alphaISO','j2','stress_var','stress_var2', 'Stress0', 'Iter_H', 'Iter_Psi', 'kappa', 'H', 'psi'])
    DriverOutput = DriverOutput(StaticParam, time, strain, stress, alphaISO_plot, j2_plot, stress_var_plot, stress_var2_plot, Stress0, Iter_H, Iter_Psi, kappa, H, psi)

    return DriverOutput
    # =========================================================================